In [4]:
## CONFIG
from datetime import datetime


naob_file = "NAOB 2001-05.xlsx"
missing_file = "NAOB_konkordans/NAOB 2001-05 mangler.xlsx"

today = datetime.today().strftime("%Y-%m-%d")
outputfile = f'NAOB_2001-2005_med_URN_{today}.xlsx'


In [3]:
## String formatting and parsing functions
from typing import Iterable
import re


def parse_publication(pub_string: str) -> dict:
    """Parse a publication string into a dictionary.

    The publication string is expected to be in the format:
        'Forfatter – Tittel: Undertittel (Sted År)'

    where 'År' is the year of publication in 4 digits (YYYY).
    """
    match = re.match(r'(.*?) – (.*?) \((.*?) (\d{4})\)', pub_string)
    if match:
        forfatter = match.group(1)
        full_tittel = match.group(2)
        sted = match.group(3)
        år = match.group(4)

        # Del opp tittel og undertittel ved siste forekomst av ': '
        tittel_deler = full_tittel.rsplit(': ', 1)
        tittel = tittel_deler[0].strip()
        undertittel = tittel_deler[1].strip() if len(tittel_deler) > 1 else None

        return {
            'Forfatter': forfatter,
            'Tittel': tittel,
            'Undertittel': undertittel,
            'Sted': sted,
            'År': år
        }
    return {}


def replace_non_alpha(string: str, strict: bool = False) -> str:
    """Replace non-alphanumeric punctuation marks with whitespace in the string."""
    if (string is None) or (not string) or (isinstance(string, float)):
        return None
    if isinstance(string, int):
        return str(string)
    if not string.isalnum():
        iterable_string = string.split() if strict else string

        for x in iterable_string:
            string = string.replace(x, " ") if not x.isalnum() else string
        #print(string)
    return string.strip()


def flip_names(author: str) -> str:
    return " ".join(reversed(author.strip().split(" , ")))


def remove_author_annotation(author: str) -> str:
    return author.replace(" (red.)", "").replace(" et al.", "")


def remove_initials(author: str) -> str:
    if author is None:
        return None
    return " ".join([x for x in author.split() if len(x) > 1])


def split_authors(authors: str, sep="/") -> Iterable[str]:
    if authors is None:
        return None
    return [auth.strip() for auth in authors.split(" " + sep + " ")]


def extract_author(authors: str, sep="/") -> Iterable[str]:
    try:
        authorlist = split_authors(authors, sep)
        first_author = authorlist[0]
        return first_author
    except TypeError:
        return None
    except AttributeError:
        return None


def format_authors_for_comparison(authors: str) -> str:
    """Format authors from dhlab.Corpus to compare with the NAOB data."""
    return [
        flip_names(auth).strip() for auth in split_authors(authors)
    ]


In [4]:
# Parse metadata from NAOB and query NB API to get URNs and persistent URLs
import pandas as pd
import  dhlab as dh
import dhlab.api.dhlab_api as api
from datetime import datetime


# Les Excel-arket
df = pd.read_excel(naob_file, names=["Publikasjon", "Årstall"])

# Anvend parsing-funksjonen på hver rad
df = pd.concat([df, df['Publikasjon'].apply(parse_publication).apply(pd.Series)], axis=1)

# Formater strengverdiene før spørring
query_df = df.copy()

# Fjern non-alfanumeriske tegn fra tittel
query_df["Tittel"] = df.Tittel.apply(replace_non_alpha)

# Fjern annotasjoner som angir flere forfattere/ roller
query_df["Forfatter"] = query_df.Forfatter.str.replace(" (red.)", "").replace(" et al.", "")

# Fjern non-alfanumeriske tegn fra forfatter
query_df["Forfatter"] = query_df.Forfatter.apply(replace_non_alpha)

# FJern initialer fra forfatter
query_df["Forfatter"] = query_df.Forfatter.apply(remove_initials)

query_df["Førsteforfatter"] = query_df.Forfatter.apply(extract_author, args=("og",))


def get_corpus_from_metadata(row):
    try:
        results = api.document_corpus(
            author=row['Førsteforfatter'],  #Søk bare på første forfatter hvis det er flere
            title=row['Tittel'],
            from_year = int(row['Årstall']),
            to_year = int(row['Årstall']) + 1,
            doctype = "digibok",
        )
    except Exception as e:
        print(f"Failed to query metadata for row {row.name}, {row['Tittel']} by {row['Forfatter']}: {e}")
        #raise e
        results = pd.DataFrame()
    return results


for idx, row in df.iterrows():
    query_row = query_df.loc[idx]
    metadata = get_corpus_from_metadata(query_row)

    if not metadata.empty:
        #urn = find_best_match(row, metadata)
        urn = metadata.iloc[0].get('urn', '')
        df.loc[idx, 'URN'] = urn
        df.loc[idx, 'URL'] = 'https://urn.nb.no/' + df.loc[idx, 'URN']

# df.URL = df.URN.map(lambda x: f"https://urn.nb.no/{x}" if x else None)

# Lagre resultatet til en ny Excel-fil
outputfile = f'NAOB_2001-2005_med_URN_{datetime.today().strftime("%Y-%m-%d")}.xlsx'
df.to_excel(outputfile, index=False)



In [9]:
# SJEKK RESULTATET
# Sjekk at forfattere med "og" er splittet riktig
# query_df[query_df.Forfatter.fillna("").str.contains("og")]

# Se på de radene som mangler forfatter -> disse må sjekkes
# Hent ut klikkbare lenker til de radene som mangler forfatter
print(df.loc[df.Forfatter.isna()].URL.to_list())

# ingen av dem stemmer overens med NAOB-tekstene, så vi må fylle inn manuelt

['https://urn.nb.no/URN:NBN:no-nb_digibok_2009032600006',
 'https://urn.nb.no/URN:NBN:no-nb_digibok_2015062508163',
 'https://urn.nb.no/URN:NBN:no-nb_digibok_2013030806002',
 'https://urn.nb.no/URN:NBN:no-nb_digibok_2016072508022']

In [12]:
# RUNDE 2

# Load a list of titles missing URNs from the last corpus batch (80)
missing = pd.read_excel("NAOB 2001-05 mangler.xlsx").rename(columns={"Kolonne A": "Publikasjon", "År": "Årstall"})

# Merge the two lists (No new titles in the missing list, length is 421)
fillgaps = df.merge(missing, how='left', on=["Publikasjon", "Årstall"])

fillgaps[(fillgaps.Grunn == "Mer enn én forfatter") & fillgaps.URL.isna()]
# Gir 1 treff: Christopher Friis-Baastad Grøndahl og Arne Svingen - Ayatollah highway

print(f"{len(fillgaps[fillgaps.URL.isna()])} titler har ikke fått URL.")

# Lagre resultatet til en ny Excel-fil
# fillgaps.to_excel("NAOB_2001-2005_med_URN_og_mangler.xlsx", index=False)

# Se på radene som mangler URL
fillgaps[fillgaps.URL.isna()]

36 titler har ikke fått URL.


Publikasjon  Årstall  \
0    Anne Enger Lahnstein – Grønn dame, rød klut : ...     2001   
11   Brynjulf Raaen – Den som brenner får svi : en ...     2001   
21    Frode Øverli – Pondus. Første omgang (Oslo 2001)     2001   
26   Hans-Wilhelm Steinfeld – Frihetens bitre tiår ...     2001   
63   Sissel Lange-Nielsen – Rød engel : en russisk ...     2001   
67   Sverre Knudsen – Spille for stjernene : roman ...     2001   
87   Anne B. Ragde – Dr. Zellwegers gave : roman (O...     2002   
100  Christopher Friis-Baastad Grøndahl og Arne Svi...     2002   
102                 Dag Solstad – 16.07.41 (Oslo 2002)     2002   
105       Espen Hammer – Theodor W. Adorno (Oslo 2002)     2002   
109  Finn-Erik Vinje – Moderne norsk : en veilednin...     2002   
114  Gunnar A. Skadberg – Alexander L. Kielland : «...     2002   
142     Lars Saabye Christensen – Sneglene (Oslo 2002)     2002   
173  Tor Bomann-Larsen – Kongstanken. Haakon & Maud...     2002   
184    Anna Bache-Wiig – Det aller fineste (Oslo 2003)     2003   
217  Jon Øystein Flink – Ole-Kristian Oksrød : roma...     2003   
255  Thure Erik Lund – Forgreininger : essays og an...     2003   
261  Tote Seierstad – I ildens rike : opplevelser l...     2003   
262       Tove Nilsen – Kreta-døgn : roman (Oslo 2003)     2003   
264     Vigdis Hjorth – 17.15 til Tønsberg (Oslo 2003)     2003   
280       Dag Solstad – Artikler 1993–2004 (Oslo 2004)     2004   
314         Liv Køltzow – Essays 1975–2004 (Oslo 2004)     2004   
327  Per Knutsen – Frøken Markussens reise : roman ...     2004   
332  Sissel Lange-Nielsen – Vår frue av Kazans hjer...     2004   
337  Tor Bomann-Larsen – Folket. Haakon & Maud II (...     2004   
339  Unni Langås – Kroppens betydning i norsk litte...     2004   
350  Arne Lyngvi – Fordi de var jøder ... : da Holo...     2005   
358  Elisabeth Botterli et al. – Madonna-gåten (Osl...     2005   
362       Erik Lundesgaard – Skikk og bruk (Oslo 2005)     2005   
367  Finn‑Erik Vinje – Norsk grammatikk : det språk...     2005   
373  Hanne Lauvstad – Helicons Bierge og Helgelands...     2005   
376  Hans-Emil Lidén – Nicolay Nicolaysen : et blad...     2005   
380  Ingvar Ambjørnsen – Elling. Samlebind (Oslo 20...     2005   
396  Mari Maurstad – Sex- og familieliv : usensurer...     2005   
400  Nazneen Khan-Østrem – Min hellige krig (Oslo 2...     2005   
402  Olav Njølstad – Brennofferet : Elie Ezerman-fi...     2005   

                                             Forfatter  \
0                                 Anne Enger Lahnstein   
11                                      Brynjulf Raaen   
21                                        Frode Øverli   
26                              Hans-Wilhelm Steinfeld   
63                                Sissel Lange-Nielsen   
67                                      Sverre Knudsen   
87                                       Anne B. Ragde   
100  Christopher Friis-Baastad Grøndahl og Arne Svi...   
102                                        Dag Solstad   
105                                       Espen Hammer   
109                                    Finn-Erik Vinje   
114                                 Gunnar A. Skadberg   
142                            Lars Saabye Christensen   
173                                  Tor Bomann-Larsen   
184                                    Anna Bache-Wiig   
217                                  Jon Øystein Flink   
255                                    Thure Erik Lund   
261                                     Tote Seierstad   
262                                        Tove Nilsen   
264                                      Vigdis Hjorth   
280                                        Dag Solstad   
314                                        Liv Køltzow   
327                                        Per Knutsen   
332                               Sissel Lange-Nielsen   
337                                  Tor Bomann-Larsen   
339                                        Unni Langås 

Jeg har gått gjennom listen manuelt, søkt opp tittel og/eller forfatter på nb.no og kopiert permanent lenke med URN. 

Lagret med nytt navn: `"NAOB_2001-2005_med_URN_og_mangler_manuelt_redigert.xlsx"`

Endring 20.08.2024: 

* Det ene av de to som var ikke funnet, heter egentlig «Angell 2002» og har varig lenke https://urn.nb.no/URN:NBN:no-nb_digibok_2009021600099
* Dag Solstad roman skal faktisk være fra 2002, men vi hadde feil tittel med punktumer istf. bindestreker. URL er da isteden https://urn.nb.no/URN:NBN:no-nb_digibok_2009042104007
* Den følgende tittelen fra 2007 kan strykes fra lista; den er ikke allment tilgjengelig, og det er en upublisert utgave fra 2005 vi har brukt:

    Hanne Lauvstad – Helicons Bierge og Helgelands schiær : Nordlands Trompets tekst, repertoar og retorikk


Dette er endret i den manuelt redigerte fila.

In [23]:
import pandas as pd

# Last inn manuelt redigerte data
manual_fixes = "NAOB_2001-2005_med_URN_og_mangler_manuelt_redigert.xlsx"

cols = "Publikasjon	Årstall	Forfatter	Tittel	Undertittel	Sted	urn	URL	Kommentar".split()
df = pd.read_excel(manual_fixes, names=cols)#.drop(index=105) # Fjern raden med kolonnenavnene

# Fjern https-prefixet fra URN-kolonnen
df.urn = df.urn.str.replace("https://urn.nb.no/", "")
df.urn = df.urn.str.replace("https://www.nb.no/items/", "")


# RUNDE 3 (20.08.2024)
# E-post: "Vedlagt er en annen liste over 50 nye titler som vi har lagt til litteraturlista vår etter forrige oppdatering av korpuset. Da kan disse titlene komme inn i korpuset samtidig."
# "Her ser Norske sølvskjeer for øvrig ut til å ha 1977 som feilaktig årstall i NBs metadata; jeg finner ikke annet enn 1974 i boka."

extra_file = "Nye4url.xlsx"
cols = "Publikasjon	URL	Årstall Kommentar".split()
extra = pd.read_excel(extra_file, names=cols)

extra["urn"] = extra.URL.str.replace("https://urn.nb.no/", "")

# Lagre til outputfil
pd.concat([df, extra], ignore_index=True).to_excel(outputfile, index=False)

In [25]:
# SISTE STEG: LAGRE NY DATA TIL NAOB.CSV

# Last inn siste lagrede fil
df = pd.read_excel(outputfile)

# Hent et dhlab.Corpus-objekt fra URNene
valid_urns = df.urn[df.urn.str.startswith("URN")]
new = valid_urns.to_frame()

# Annoter batch for URNene
new["corpus"] = "naob_2024"
#new.rename(columns={"URN": "urn"}, inplace=True)

# Kombiner med eksisterende fil og lagre
naob_corpus = pd.read_csv("naob.csv", index_col=0)
pd.concat([naob_corpus, new]).to_csv("naob.csv")

In [259]:
# IKKE FERDIG - MÅ FIKSE LOGIKKEN

# Funksjon for å finne beste treff
def find_best_match(row, metadata):

    author=row['Forfatter']
    title=row['Tittel']
    year = int(row['Årstall'])


#    search_string = row['Søkestreng'].lower()
    best_match = None
    best_score = 0

    for idx, item in metadata.iterrows():
        dh_title = item.get('title', '').lower()
        authors = ' '.join(item.get('authors', [])).lower()
        year = str(item.get('year', ''))

        #item_string = f"{title} {authors} {year}"

        # Enkel poengberegning basert på hvor mange ord som matcher
        score = sum(1 for word in search_string.split() if word in item_string)

        if score > best_score:
            best_score = score
            best_match = item

    return best_match['urn'] if best_match else ''




In [23]:
## Test parse_publication
examples = [
    "Forfatter – Hovedtittel: Undertittel (Oslo 2001)",
    "Forfatter1 og Forfatter2 – Hovedtittel: Undertittel (Oslo 2001)",
    "Forfatter – Kompleks tittel: med flere: kolon: Faktisk undertittel (Oslo 2001)",
    "Forfatter – Tittel uten undertittel (Oslo 2001)"
]

for example in examples:
    result = parse_publication(example)
    print(f"Original: {example}")
    print(f"Parsed: {result}\n")


Original: Forfatter – Hovedtittel: Undertittel (Oslo 2001)
Parsed: {'Forfatter': 'Forfatter', 'Tittel': 'Hovedtittel', 'Undertittel': 'Undertittel', 'Sted': 'Oslo', 'År': '2001'}

Original: Forfatter1 og Forfatter2 – Hovedtittel: Undertittel (Oslo 2001)
Parsed: {'Forfatter': 'Forfatter1 og Forfatter2', 'Tittel': 'Hovedtittel', 'Undertittel': 'Undertittel', 'Sted': 'Oslo', 'År': '2001'}

Original: Forfatter – Kompleks tittel: med flere: kolon: Faktisk undertittel (Oslo 2001)
Parsed: {'Forfatter': 'Forfatter', 'Tittel': 'Kompleks tittel: med flere: kolon', 'Undertittel': 'Faktisk undertittel', 'Sted': 'Oslo', 'År': '2001'}

Original: Forfatter – Tittel uten undertittel (Oslo 2001)
Parsed: {'Forfatter': 'Forfatter', 'Tittel': 'Tittel uten undertittel', 'Undertittel': None, 'Sted': 'Oslo', 'År': '2001'}

